# 🎯 INTELLIGENT SUPPLY CHAIN COORDINATION SYSTEM
## Mitigating Bullwhip Effect using ML/DL/RL

---

**Problem Statement:**  
Lack of effective coordination among supply chain partners leads to distorted demand information, increased variability, late deliveries, excess inventory, and inefficient operations across the supply chain. Traditional supply chain systems operate in isolated stages where forecasting, ordering, and replenishment decisions are taken independently, resulting in the bullwhip effect and poor synchronization between supply and demand.

**Solution:**  
An intelligent coordination system that can accurately forecast demand, predict delivery risks, and support adaptive decision-making to reduce variability and improve overall supply chain performance.

---

### 📋 Table of Contents:
1. Setup & Data Loading
2. Exploratory Data Analysis (EDA)
3. Feature Engineering
4. Bullwhip Effect Analysis
5. Demand Forecasting (LSTM)
6. Delivery Risk Prediction (XGBoost)
7. Geographic Analysis & Clustering
8. Reinforcement Learning (DQN)
9. Coordination Recommendations
10. Model Saving & Results

---

# ═══════════════════════════════════════════════════════════
# SECTION 1: SETUP & DATA LOADING
# ═══════════════════════════════════════════════════════════

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy matplotlib seaborn plotly folium xgboost scikit-learn tensorflow imbalanced-learn networkx prophet tqdm

print("✅ All packages installed successfully!")

In [ ]:
# ═══════════════════════════════════════════════════════════
# 1.1 IMPORT LIBRARIES
# ═══════════════════════════════════════════════════════════

# Data Processing
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    confusion_matrix, classification_report, roc_auc_score, roc_curve,
    mean_squared_error, mean_absolute_error, r2_score
)
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from imblearn.over_sampling import SMOTE

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# XGBoost
import xgboost as xgb

# Model Saving
import pickle
import joblib

# Progress Bar
from tqdm.notebook import tqdm

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("\n" + "="*60)
print("✅ ALL LIBRARIES IMPORTED SUCCESSFULLY!")
print("="*60)
print(f"TensorFlow Version: {tf.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")
print("="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════
# 1.2 LOAD DATACO DATASET
# ═══════════════════════════════════════════════════════════

# IMPORTANT: Upload your DataCo CSV file to Colab first!
# Click on the folder icon on the left -> Upload

# Option 1: If file is in Colab root directory
FILE_PATH = 'DataCoSupplyChainDataset.csv'  # Change this to your filename

# Option 2: If using Google Drive (uncomment below)
# from google.colab import drive
# drive.mount('/content/drive')
# FILE_PATH = '/content/drive/MyDrive/DataCoSupplyChainDataset.csv'

print("Loading DataCo Supply Chain Dataset...\n")

try:
    # Load the dataset
    df = pd.read_csv(FILE_PATH, encoding='latin-1')  # Try latin-1 if utf-8 fails
    
    print("="*60)
    print("✅ DATASET LOADED SUCCESSFULLY!")
    print("="*60)
    print(f"\n📊 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"\n📋 Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
except FileNotFoundError:
    print("❌ ERROR: File not found!")
    print("\nPlease upload the DataCo dataset CSV file to Colab.")
    print("Instructions:")
    print("1. Click the folder icon on the left sidebar")
    print("2. Click the upload button")
    print("3. Select your DataCoSupplyChainDataset.csv file")
    print("4. Update FILE_PATH variable above if needed")
    raise

In [ ]:
# ═══════════════════════════════════════════════════════════
# 1.3 INITIAL DATA OVERVIEW
# ═══════════════════════════════════════════════════════════

print("\n" + "="*60)
print("📋 COLUMN NAMES")
print("="*60)
for i, col in enumerate(df.columns, 1):
    print(f"{i:3d}. {col}")

print("\n" + "="*60)
print("🔍 FIRST 5 ROWS")
print("="*60)
display(df.head())

print("\n" + "="*60)
print("📊 DATA TYPES")
print("="*60)
print(df.dtypes)

print("\n" + "="*60)
print("📈 BASIC STATISTICS")
print("="*60)
display(df.describe())

# ═══════════════════════════════════════════════════════════
# SECTION 2: EXPLORATORY DATA ANALYSIS (EDA)
# ═══════════════════════════════════════════════════════════

In [ ]:
# ═══════════════════════════════════════════════════════════
# 2.1 DATA CLEANING & PREPROCESSING
# ═══════════════════════════════════════════════════════════

print("Starting data cleaning...\n")

# Store original shape
original_shape = df.shape

# 1. Check for missing values
print("="*60)
print("🔍 MISSING VALUES ANALYSIS")
print("="*60)
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing_values,
    'Percentage': missing_percent
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
    
    # Drop columns with >50% missing values
    cols_to_drop = missing_df[missing_df['Percentage'] > 50].index.tolist()
    if cols_to_drop:
        print(f"\n⚠️  Dropping columns with >50% missing: {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)
    
    # Fill remaining missing values
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            if df[col].dtype in ['float64', 'int64']:
                df[col].fillna(df[col].median(), inplace=True)
            else:
                df[col].fillna(df[col].mode()[0], inplace=True)
else:
    print("✅ No missing values found!")

# 2. Check for duplicates
duplicates = df.duplicated().sum()
print(f"\n🔍 Duplicate Rows: {duplicates}")
if duplicates > 0:
    df = df.drop_duplicates()
    print(f"✅ Removed {duplicates} duplicate rows")

# 3. Standardize column names (remove spaces, lowercase)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

print(f"\n📊 Shape after cleaning: {df.shape}")
print(f"📊 Original shape: {original_shape}")
print(f"✅ Rows removed: {original_shape[0] - df.shape[0]}")

# Display cleaned data info
print("\n" + "="*60)
print("📋 CLEANED DATA INFO")
print("="*60)
df.info()

In [ ]:
# ═══════════════════════════════════════════════════════════
# 2.2 KEY METRICS OVERVIEW
# ═══════════════════════════════════════════════════════════

# Identify key columns (adjust based on actual column names)
# Common DataCo columns:
key_cols = {
    'order_item_quantity': 'Order Quantity',
    'sales': 'Sales',
    'late_delivery_risk': 'Late Delivery Risk',
    'days_for_shipping_real': 'Actual Shipping Days',
    'days_for_shipment_scheduled': 'Scheduled Shipping Days',
    'order_status': 'Order Status',
    'shipping_mode': 'Shipping Mode',
    'market': 'Market',
    'order_region': 'Region',
    'customer_segment': 'Customer Segment',
    'category_name': 'Category',
}

# Check which columns exist in the dataset
existing_cols = {k: v for k, v in key_cols.items() if k in df.columns}

print("="*60)
print("🎯 KEY SUPPLY CHAIN METRICS")
print("="*60)

if 'order_item_quantity' in df.columns:
    print(f"\n📦 Total Orders: {len(df):,}")
    print(f"📦 Total Quantity Ordered: {df['order_item_quantity'].sum():,.0f}")
    print(f"📦 Avg Order Quantity: {df['order_item_quantity'].mean():.2f}")

if 'sales' in df.columns:
    print(f"\n💰 Total Sales: ${df['sales'].sum():,.2f}")
    print(f"💰 Avg Sales per Order: ${df['sales'].mean():.2f}")

if 'late_delivery_risk' in df.columns:
    late_risk = df['late_delivery_risk'].value_counts()
    print(f"\n🚚 Late Delivery Risk:")
    print(f"   - High Risk: {late_risk.get(1, 0):,} ({late_risk.get(1, 0)/len(df)*100:.1f}%)")
    print(f"   - Low Risk: {late_risk.get(0, 0):,} ({late_risk.get(0, 0)/len(df)*100:.1f}%)")

if 'days_for_shipping_real' in df.columns and 'days_for_shipment_scheduled' in df.columns:
    df['shipping_delay'] = df['days_for_shipping_real'] - df['days_for_shipment_scheduled']
    print(f"\n⏱️  Shipping Performance:")
    print(f"   - Avg Actual Days: {df['days_for_shipping_real'].mean():.2f}")
    print(f"   - Avg Scheduled Days: {df['days_for_shipment_scheduled'].mean():.2f}")
    print(f"   - Avg Delay: {df['shipping_delay'].mean():.2f} days")

if 'market' in df.columns:
    print(f"\n🌍 Markets Served: {df['market'].nunique()}")
    print(df['market'].value_counts())

if 'order_region' in df.columns:
    print(f"\n📍 Regions: {df['order_region'].nunique()}")

if 'shipping_mode' in df.columns:
    print(f"\n🚛 Shipping Modes:")
    print(df['shipping_mode'].value_counts())

In [ ]:
# ═══════════════════════════════════════════════════════════
# 2.3 VISUALIZATIONS - DISTRIBUTION ANALYSIS
# ═══════════════════════════════════════════════════════════

# Create subplots for key distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Supply Chain Key Metrics Distribution', fontsize=16, fontweight='bold')

# 1. Order Quantity Distribution
if 'order_item_quantity' in df.columns:
    axes[0, 0].hist(df['order_item_quantity'], bins=50, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Order Quantity Distribution')
    axes[0, 0].set_xlabel('Quantity')
    axes[0, 0].set_ylabel('Frequency')

# 2. Sales Distribution
if 'sales' in df.columns:
    axes[0, 1].hist(df['sales'], bins=50, color='lightgreen', edgecolor='black')
    axes[0, 1].set_title('Sales Distribution')
    axes[0, 1].set_xlabel('Sales ($)')
    axes[0, 1].set_ylabel('Frequency')

# 3. Late Delivery Risk
if 'late_delivery_risk' in df.columns:
    risk_counts = df['late_delivery_risk'].value_counts()
    axes[0, 2].bar(['Low Risk', 'High Risk'], risk_counts.values, color=['green', 'red'])
    axes[0, 2].set_title('Late Delivery Risk')
    axes[0, 2].set_ylabel('Count')

# 4. Shipping Days - Real vs Scheduled
if 'days_for_shipping_real' in df.columns and 'days_for_shipment_scheduled' in df.columns:
    axes[1, 0].hist(df['days_for_shipping_real'], bins=30, alpha=0.7, label='Actual', color='orange')
    axes[1, 0].hist(df['days_for_shipment_scheduled'], bins=30, alpha=0.7, label='Scheduled', color='blue')
    axes[1, 0].set_title('Shipping Days: Actual vs Scheduled')
    axes[1, 0].set_xlabel('Days')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].legend()

# 5. Shipping Delay Distribution
if 'shipping_delay' in df.columns:
    axes[1, 1].hist(df['shipping_delay'], bins=50, color='coral', edgecolor='black')
    axes[1, 1].set_title('Shipping Delay Distribution')
    axes[1, 1].set_xlabel('Delay (days)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].axvline(x=0, color='red', linestyle='--', label='On Time')
    axes[1, 1].legend()

# 6. Market Distribution
if 'market' in df.columns:
    market_counts = df['market'].value_counts().head(10)
    axes[1, 2].barh(market_counts.index, market_counts.values, color='purple')
    axes[1, 2].set_title('Top Markets')
    axes[1, 2].set_xlabel('Count')

plt.tight_layout()
plt.show()

print("✅ Distribution visualizations complete!")

In [ ]:
# ═══════════════════════════════════════════════════════════
# 2.4 CORRELATION ANALYSIS
# ═══════════════════════════════════════════════════════════

# Select numerical columns for correlation
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remove ID columns
numerical_cols = [col for col in numerical_cols if 'id' not in col.lower()]

if len(numerical_cols) > 1:
    # Calculate correlation matrix
    correlation_matrix = df[numerical_cols].corr()
    
    # Create correlation heatmap
    plt.figure(figsize=(14, 10))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Find strong correlations (>0.7 or <-0.7)
    print("\n" + "="*60)
    print("🔍 STRONG CORRELATIONS (|r| > 0.7)")
    print("="*60)
    
    strong_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.7:
                strong_corr.append((
                    correlation_matrix.columns[i],
                    correlation_matrix.columns[j],
                    correlation_matrix.iloc[i, j]
                ))
    
    if strong_corr:
        for var1, var2, corr in sorted(strong_corr, key=lambda x: abs(x[2]), reverse=True):
            print(f"{var1} <-> {var2}: {corr:.3f}")
    else:
        print("No strong correlations found.")
else:
    print("⚠️  Not enough numerical columns for correlation analysis")

# ═══════════════════════════════════════════════════════════
# SECTION 3: FEATURE ENGINEERING
# ═══════════════════════════════════════════════════════════

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.1 DATE/TIME FEATURES EXTRACTION
# ═══════════════════════════════════════════════════════════

print("Creating date/time features...\n")

# Find date columns
date_columns = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
print(f"Date columns found: {date_columns}")

# Process order date if exists
if 'order_date_dateorders' in df.columns or 'order_date' in df.columns:
    date_col = 'order_date_dateorders' if 'order_date_dateorders' in df.columns else 'order_date'
    
    # Convert to datetime
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    
    # Extract features
    df['order_year'] = df[date_col].dt.year
    df['order_month'] = df[date_col].dt.month
    df['order_day'] = df[date_col].dt.day
    df['order_dayofweek'] = df[date_col].dt.dayofweek  # 0=Monday, 6=Sunday
    df['order_quarter'] = df[date_col].dt.quarter
    df['order_week'] = df[date_col].dt.isocalendar().week
    df['is_weekend'] = df['order_dayofweek'].isin([5, 6]).astype(int)
    df['is_month_start'] = df[date_col].dt.is_month_start.astype(int)
    df['is_month_end'] = df[date_col].dt.is_month_end.astype(int)
    
    print(f"\n✅ Extracted date features from '{date_col}'")
    print(f"   - Year, Month, Day, Day of Week, Quarter, Week")
    print(f"   - Weekend flag, Month start/end flags")
    print(f"\n📅 Date Range: {df[date_col].min()} to {df[date_col].max()}")
    print(f"📅 Total Days: {(df[date_col].max() - df[date_col].min()).days}")
else:
    print("⚠️  No order date column found")

# Process shipping date if exists
if 'shipping_date_dateorders' in df.columns or 'shipping_date' in df.columns:
    ship_date_col = 'shipping_date_dateorders' if 'shipping_date_dateorders' in df.columns else 'shipping_date'
    df[ship_date_col] = pd.to_datetime(df[ship_date_col], errors='coerce')
    print(f"\n✅ Processed shipping date: {ship_date_col}")

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.2 SUPPLY CHAIN TIER FEATURES
# ═══════════════════════════════════════════════════════════

print("Creating supply chain tier features...\n")

# Define supply chain hierarchy
# Tier 1: Customer/Retailer (most downstream)
# Tier 2: Department/Store
# Tier 3: Market
# Tier 4: Region (most upstream)

tier_mapping = {
    'customer_segment': 'tier_1_customer',
    'department_name': 'tier_2_department',
    'market': 'tier_3_market',
    'order_region': 'tier_4_region'
}

# Create tier features
for original_col, new_col in tier_mapping.items():
    if original_col in df.columns:
        df[new_col] = df[original_col]
        print(f"✅ Created {new_col} from {original_col}")

# Create aggregated tier identifier
tier_cols = [v for k, v in tier_mapping.items() if k in df.columns]
if tier_cols:
    df['supply_chain_path'] = df[tier_cols].astype(str).agg('_'.join, axis=1)
    print(f"\n✅ Created supply_chain_path combining all tiers")
    print(f"   Unique paths: {df['supply_chain_path'].nunique()}")

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.3 VARIABILITY METRICS
# ═══════════════════════════════════════════════════════════

print("Calculating variability metrics...\n")

if 'order_item_quantity' in df.columns:
    # Calculate rolling statistics for order quantity
    # Group by customer and calculate rolling metrics
    
    if 'customer_id' in df.columns and 'order_date_dateorders' in df.columns:
        # Sort by customer and date
        df_sorted = df.sort_values(['customer_id', 'order_date_dateorders'])
        
        # Calculate rolling mean and std (window=3)
        df_sorted['quantity_rolling_mean_3'] = df_sorted.groupby('customer_id')['order_item_quantity'].transform(
            lambda x: x.rolling(window=3, min_periods=1).mean()
        )
        df_sorted['quantity_rolling_std_3'] = df_sorted.groupby('customer_id')['order_item_quantity'].transform(
            lambda x: x.rolling(window=3, min_periods=1).std()
        )
        
        # Coefficient of variation
        df_sorted['quantity_cv'] = df_sorted['quantity_rolling_std_3'] / (df_sorted['quantity_rolling_mean_3'] + 1e-6)
        
        # Merge back to original df
        df = df_sorted
        
        print("✅ Created rolling variability metrics:")
        print("   - quantity_rolling_mean_3")
        print("   - quantity_rolling_std_3")
        print("   - quantity_cv (coefficient of variation)")
    
    # Calculate order size categories
    df['order_size_category'] = pd.cut(
        df['order_item_quantity'],
        bins=[0, 1, 3, 5, 10, float('inf')],
        labels=['Very Small', 'Small', 'Medium', 'Large', 'Very Large']
    )
    print("\n✅ Created order_size_category")
    print(df['order_size_category'].value_counts())

# Calculate discount impact
if 'order_item_discount_rate' in df.columns:
    df['has_discount'] = (df['order_item_discount_rate'] > 0).astype(int)
    df['discount_category'] = pd.cut(
        df['order_item_discount_rate'],
        bins=[-0.01, 0, 0.1, 0.2, 0.5, 1.0],
        labels=['No Discount', 'Low (0-10%)', 'Medium (10-20%)', 'High (20-50%)', 'Very High (>50%)']
    )
    print("\n✅ Created discount features")
    print(df['discount_category'].value_counts())

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.4 CATEGORICAL ENCODING
# ═══════════════════════════════════════════════════════════

print("Encoding categorical variables...\n")

# Create label encoders dictionary to save for later use
label_encoders = {}

# Categorical columns to encode
categorical_cols_to_encode = [
    'shipping_mode',
    'customer_segment',
    'market',
    'order_region',
    'category_name',
    'department_name',
    'order_status',
    'delivery_status'
]

# Filter to only existing columns
categorical_cols_to_encode = [col for col in categorical_cols_to_encode if col in df.columns]

# Apply label encoding
for col in categorical_cols_to_encode:
    le = LabelEncoder()
    df[f'{col}_encoded'] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le
    print(f"✅ Encoded {col} -> {col}_encoded ({df[col].nunique()} categories)")

print(f"\n✅ Total categorical features encoded: {len(categorical_cols_to_encode)}")
print(f"\n📊 Current DataFrame shape: {df.shape}")

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.5 FEATURE SUMMARY
# ═══════════════════════════════════════════════════════════

print("="*60)
print("📊 FEATURE ENGINEERING SUMMARY")
print("="*60)

new_features = [col for col in df.columns if any([
    col.startswith('order_'),
    col.startswith('tier_'),
    col.startswith('quantity_'),
    col.endswith('_encoded'),
    col.endswith('_category'),
    col in ['is_weekend', 'has_discount', 'shipping_delay', 'supply_chain_path']
])]

print(f"\n✅ Total new features created: {len(new_features)}")
print("\nNew feature categories:")
print(f"  - Date/Time features: {len([f for f in new_features if f.startswith('order_')])}")
print(f"  - Supply chain tiers: {len([f for f in new_features if f.startswith('tier_')])}")
print(f"  - Variability metrics: {len([f for f in new_features if f.startswith('quantity_')])}")
print(f"  - Encoded features: {len([f for f in new_features if f.endswith('_encoded')])}")
print(f"  - Category features: {len([f for f in new_features if f.endswith('_category')])}")

print(f"\n📊 Final DataFrame shape: {df.shape}")
print("\n✅ Feature engineering complete!")

# Save processed dataframe
df.to_csv('processed_supply_chain_data.csv', index=False)
print("\n💾 Saved processed data to 'processed_supply_chain_data.csv'")

# ═══════════════════════════════════════════════════════════
# SECTION 4: BULLWHIP EFFECT ANALYSIS
# ═══════════════════════════════════════════════════════════

In [ ]:
# ═══════════════════════════════════════════════════════════
# 4.1 MULTI-TIER VARIANCE CALCULATION
# ═══════════════════════════════════════════════════════════

print("Calculating Bullwhip Effect across supply chain tiers...\n")

def calculate_bullwhip_effect(df, tiers, quantity_col='order_item_quantity'):
    """
    Calculate variance amplification across supply chain tiers
    
    Parameters:
    -----------
    df : DataFrame
        Supply chain data
    tiers : list
        List of tier column names from downstream to upstream
    quantity_col : str
        Column name for order quantity
        
    Returns:
    --------
    results : dict
        Dictionary containing variance metrics per tier
    """
    results = {}
    
    for tier in tiers:
        if tier in df.columns:
            # Aggregate orders by tier
            tier_data = df.groupby(tier)[quantity_col].agg([
                ('mean', 'mean'),
                ('std', 'std'),
                ('var', 'var'),
                ('count', 'count'),
                ('total', 'sum')
            ]).reset_index()
            
            # Calculate coefficient of variation
            tier_data['cv'] = tier_data['std'] / (tier_data['mean'] + 1e-6)
            
            results[tier] = {
                'data': tier_data,
                'overall_mean': df[quantity_col].mean(),
                'overall_std': df[quantity_col].std(),
                'overall_var': df[quantity_col].var(),
                'overall_cv': df[quantity_col].std() / (df[quantity_col].mean() + 1e-6),
                'num_entities': tier_data.shape[0]
            }
    
    return results

# Define supply chain tiers (from downstream to upstream)
tiers = [
    'tier_1_customer',      # Most downstream (retailer/customer level)
    'tier_2_department',    # Department level
    'tier_3_market',        # Market level
    'tier_4_region'         # Most upstream (regional level)
]

# Filter to existing tiers
existing_tiers = [t for t in tiers if t in df.columns]

if existing_tiers and 'order_item_quantity' in df.columns:
    # Calculate bullwhip effect
    bullwhip_results = calculate_bullwhip_effect(df, existing_tiers)
    
    # Display results
    print("="*60)
    print("🌊 BULLWHIP EFFECT ANALYSIS RESULTS")
    print("="*60)
    
    for tier in existing_tiers:
        if tier in bullwhip_results:
            result = bullwhip_results[tier]
            print(f"\n{'='*60}")
            print(f"📍 {tier.upper()}")
            print(f"{'='*60}")
            print(f"Number of entities: {result['num_entities']}")
            print(f"Overall Mean: {result['overall_mean']:.2f}")
            print(f"Overall Std Dev: {result['overall_std']:.2f}")
            print(f"Overall Variance: {result['overall_var']:.2f}")
            print(f"Coefficient of Variation: {result['overall_cv']:.4f}")
    
    # Calculate amplification ratios between tiers
    print(f"\n{'='*60}")
    print("📊 VARIANCE AMPLIFICATION RATIOS")
    print(f"{'='*60}")
    
    amplification_ratios = []
    for i in range(len(existing_tiers) - 1):
        downstream_tier = existing_tiers[i]
        upstream_tier = existing_tiers[i + 1]
        
        if downstream_tier in bullwhip_results and upstream_tier in bullwhip_results:
            downstream_var = bullwhip_results[downstream_tier]['overall_var']
            upstream_var = bullwhip_results[upstream_tier]['overall_var']
            
            ratio = upstream_var / (downstream_var + 1e-6)
            amplification_ratios.append({
                'from': downstream_tier,
                'to': upstream_tier,
                'ratio': ratio
            })
            
            print(f"\n{downstream_tier} → {upstream_tier}")
            print(f"  Downstream Variance: {downstream_var:.2f}")
            print(f"  Upstream Variance: {upstream_var:.2f}")
            print(f"  🌊 Amplification Ratio: {ratio:.2f}x")
            
            if ratio > 1.5:
                print(f"  ⚠️  HIGH BULLWHIP EFFECT DETECTED!")
            elif ratio > 1.0:
                print(f"  ⚡ Moderate bullwhip effect")
            else:
                print(f"  ✅ Low bullwhip effect")
    
    # Calculate total amplification
    if amplification_ratios:
        total_amplification = np.prod([r['ratio'] for r in amplification_ratios])
        print(f"\n{'='*60}")
        print(f"🎯 TOTAL AMPLIFICATION: {total_amplification:.2f}x")
        print(f"{'='*60}")
        print(f"\nThis means orders at the most upstream tier ({existing_tiers[-1]})")
        print(f"have {total_amplification:.2f}x the variance of customer demand.")
else:
    print("⚠️  Required columns not found for bullwhip analysis")

In [ ]:
# ═══════════════════════════════════════════════════════════
# 4.2 VISUALIZATION - VARIANCE BY TIER
# ═══════════════════════════════════════════════════════════

if existing_tiers and 'order_item_quantity' in df.columns:
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Variance by Tier
    tier_names = []
    variances = []
    std_devs = []
    cvs = []
    
    for tier in existing_tiers:
        if tier in bullwhip_results:
            tier_names.append(tier.replace('tier_', '').replace('_', ' ').title())
            variances.append(bullwhip_results[tier]['overall_var'])
            std_devs.append(bullwhip_results[tier]['overall_std'])
            cvs.append(bullwhip_results[tier]['overall_cv'])
    
    # Variance plot
    x_pos = np.arange(len(tier_names))
    axes[0].bar(x_pos, variances, color=['green', 'yellow', 'orange', 'red'][:len(tier_names)],
                edgecolor='black', linewidth=1.5)
    axes[0].set_xlabel('Supply Chain Tier', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Variance', fontsize=12, fontweight='bold')
    axes[0].set_title('Order Quantity Variance by Supply Chain Tier\n(Bullwhip Effect Visualization)',
                      fontsize=14, fontweight='bold')
    axes[0].set_xticks(x_pos)
    axes[0].set_xticklabels(tier_names, rotation=45, ha='right')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(variances):
        axes[0].text(i, v, f'{v:.1f}', ha='center', va='bottom', fontweight='bold')
    
    # Coefficient of Variation plot
    axes[1].plot(tier_names, cvs, marker='o', linewidth=2, markersize=10, color='red')
    axes[1].fill_between(range(len(tier_names)), cvs, alpha=0.3, color='red')
    axes[1].set_xlabel('Supply Chain Tier', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Coefficient of Variation (CV)', fontsize=12, fontweight='bold')
    axes[1].set_title('Demand Variability Amplification\n(Higher CV = More Bullwhip)',
                      fontsize=14, fontweight='bold')
    axes[1].set_xticklabels(tier_names, rotation=45, ha='right')
    axes[1].grid(alpha=0.3)
    
    # Add value labels
    for i, cv in enumerate(cvs):
        axes[1].text(i, cv, f'{cv:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('bullwhip_effect_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Bullwhip effect visualizations created!")
    print("💾 Saved as 'bullwhip_effect_analysis.png'")

In [ ]:
# ═══════════════════════════════════════════════════════════
# 4.3 ANOMALY DETECTION - ISOLATION FOREST
# ═══════════════════════════════════════════════════════════

print("Detecting demand anomalies using Isolation Forest...\n")

if 'order_item_quantity' in df.columns:
    # Prepare features for anomaly detection
    anomaly_features = ['order_item_quantity']
    
    # Add more features if available
    optional_features = ['sales', 'order_item_discount_rate', 'days_for_shipping_real']
    anomaly_features.extend([f for f in optional_features if f in df.columns])
    
    # Create feature matrix
    X_anomaly = df[anomaly_features].copy()
    
    # Handle missing values
    X_anomaly = X_anomaly.fillna(X_anomaly.median())
    
    # Fit Isolation Forest
    iso_forest = IsolationForest(
        contamination=0.05,  # Expect 5% anomalies
        random_state=42,
        n_estimators=100
    )
    
    # Predict anomalies (-1 = anomaly, 1 = normal)
    df['is_anomaly'] = iso_forest.fit_predict(X_anomaly)
    df['anomaly_score'] = iso_forest.score_samples(X_anomaly)
    
    # Analysis
    n_anomalies = (df['is_anomaly'] == -1).sum()
    pct_anomalies = (n_anomalies / len(df)) * 100
    
    print("="*60)
    print("🔍 ANOMALY DETECTION RESULTS")
    print("="*60)
    print(f"Total orders analyzed: {len(df):,}")
    print(f"Anomalies detected: {n_anomalies:,} ({pct_anomalies:.2f}%)")
    print(f"Normal orders: {(df['is_anomaly'] == 1).sum():,}")
    
    # Analyze anomalies
    anomalies_df = df[df['is_anomaly'] == -1]
    
    print("\n📊 Anomaly Characteristics:")
    print(f"  Avg Quantity (Anomalies): {anomalies_df['order_item_quantity'].mean():.2f}")
    print(f"  Avg Quantity (Normal): {df[df['is_anomaly'] == 1]['order_item_quantity'].mean():.2f}")
    
    if 'sales' in df.columns:
        print(f"  Avg Sales (Anomalies): ${anomalies_df['sales'].mean():.2f}")
        print(f"  Avg Sales (Normal): ${df[df['is_anomaly'] == 1]['sales'].mean():.2f}")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Anomaly Score Distribution
    axes[0].hist(df['anomaly_score'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0].axvline(x=df[df['is_anomaly'] == -1]['anomaly_score'].max(), 
                    color='red', linestyle='--', linewidth=2, label='Anomaly Threshold')
    axes[0].set_xlabel('Anomaly Score', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[0].set_title('Anomaly Score Distribution', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Plot 2: Order Quantity - Normal vs Anomaly
    normal_qty = df[df['is_anomaly'] == 1]['order_item_quantity']
    anomaly_qty = df[df['is_anomaly'] == -1]['order_item_quantity']
    
    axes[1].hist(normal_qty, bins=50, alpha=0.7, label='Normal', color='green', edgecolor='black')
    axes[1].hist(anomaly_qty, bins=50, alpha=0.7, label='Anomaly', color='red', edgecolor='black')
    axes[1].set_xlabel('Order Quantity', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[1].set_title('Order Quantity Distribution: Normal vs Anomalies', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('anomaly_detection_results.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Anomaly detection complete!")
    print("💾 Saved visualization as 'anomaly_detection_results.png'")
    
    # Save anomalies for further analysis
    anomalies_df.to_csv('detected_anomalies.csv', index=False)
    print("💾 Saved anomalies to 'detected_anomalies.csv'")
else:
    print("⚠️  order_item_quantity column not found")

# ═══════════════════════════════════════════════════════════
# SECTION 5: DEMAND FORECASTING (LSTM)
# ═══════════════════════════════════════════════════════════

In [ ]:
# ═══════════════════════════════════════════════════════════
# 5.1 DATA PREPARATION FOR TIME SERIES
# ═══════════════════════════════════════════════════════════

print("Preparing data for LSTM demand forecasting...\n")

# Check required columns
date_col = None
for col in df.columns:
    if 'order_date' in col.lower() and 'dateorders' in col.lower():
        date_col = col
        break

if date_col and 'order_item_quantity' in df.columns:
    # Create daily aggregated data
    df_ts = df.copy()
    df_ts[date_col] = pd.to_datetime(df_ts[date_col])
    df_ts = df_ts.sort_values(date_col)
    
    # Aggregate by date
    daily_demand = df_ts.groupby(date_col).agg({
        'order_item_quantity': 'sum',
        'sales': 'sum' if 'sales' in df_ts.columns else 'count',
        'order_id': 'count'
    }).reset_index()
    
    daily_demand.columns = ['date', 'total_quantity', 'total_sales', 'num_orders']
    
    # Fill missing dates
    date_range = pd.date_range(start=daily_demand['date'].min(), 
                                end=daily_demand['date'].max(), 
                                freq='D')
    daily_demand = daily_demand.set_index('date').reindex(date_range, fill_value=0).reset_index()
    daily_demand.columns = ['date', 'total_quantity', 'total_sales', 'num_orders']
    
    print("="*60)
    print("📊 TIME SERIES DATA SUMMARY")
    print("="*60)
    print(f"Date range: {daily_demand['date'].min()} to {daily_demand['date'].max()}")
    print(f"Total days: {len(daily_demand)}")
    print(f"Total demand: {daily_demand['total_quantity'].sum():,.0f}")
    print(f"Average daily demand: {daily_demand['total_quantity'].mean():.2f}")
    print(f"Max daily demand: {daily_demand['total_quantity'].max():.0f}")
    print(f"Min daily demand: {daily_demand['total_quantity'].min():.0f}")
    
    # Visualize time series
    fig, axes = plt.subplots(2, 1, figsize=(16, 10))
    
    # Daily demand plot
    axes[0].plot(daily_demand['date'], daily_demand['total_quantity'], linewidth=1.5, color='blue')
    axes[0].set_xlabel('Date', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Total Quantity', fontsize=12, fontweight='bold')
    axes[0].set_title('Daily Demand Over Time', fontsize=14, fontweight='bold')
    axes[0].grid(alpha=0.3)
    
    # Rolling average (7-day)
    daily_demand['rolling_mean_7'] = daily_demand['total_quantity'].rolling(window=7).mean()
    daily_demand['rolling_mean_30'] = daily_demand['total_quantity'].rolling(window=30).mean()
    
    axes[1].plot(daily_demand['date'], daily_demand['total_quantity'], 
                 linewidth=1, color='lightblue', alpha=0.5, label='Daily')
    axes[1].plot(daily_demand['date'], daily_demand['rolling_mean_7'], 
                 linewidth=2, color='orange', label='7-day MA')
    axes[1].plot(daily_demand['date'], daily_demand['rolling_mean_30'], 
                 linewidth=2, color='red', label='30-day MA')
    axes[1].set_xlabel('Date', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Total Quantity', fontsize=12, fontweight='bold')
    axes[1].set_title('Demand with Moving Averages', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('time_series_demand.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Time series visualization created!")
else:
    print("⚠️  Required columns for time series analysis not found")
    daily_demand = None